In [0]:
# Databricks notebook source
# 1. Завантаження очищених даних
delta_path = "/Volumes/workspace/default/olist_delta_dataset"

olist_orders = spark.read.format("delta").load(delta_path)
olist_orders.show(5)
olist_orders.printSchema()



In [0]:
# 2. Первинний аналіз змінних
# Числові змінні
numeric_cols = ["purchase_year", "purchase_month", "purchase_day", 
                "purchase_weekday", "purchase_weekofyear", "delivery_time_days"]

# Категоріальні змінні
categorical_cols = ["order_status", "customer_id"]

# Часові змінні
time_cols = ["order_purchase_timestamp", "order_approved_at",
             "order_delivered_carrier_date", "order_delivered_customer_date",
             "order_estimated_delivery_date"]

print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)
print("Time:", time_cols)



In [0]:
# Масштабування числових ознак
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Об’єднання числових ознак у вектор
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="numeric_features")

# Стандартизація
scaler = StandardScaler(inputCol="numeric_features", outputCol="numeric_scaled", withMean=True, withStd=True)

pipeline_numeric = Pipeline(stages=[assembler, scaler])
olist_orders_scaled = pipeline_numeric.fit(olist_orders).transform(olist_orders)

olist_orders_scaled.select("numeric_features", "numeric_scaled").show(5, truncate=False)


In [0]:
# Кодування категоріальних ознак
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Індексування
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index") for col in categorical_cols]

# One-hot енкодінг
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_ohe") for col in categorical_cols]

pipeline_cat = Pipeline(stages=indexers + encoders)
olist_orders_cat = pipeline_cat.fit(olist_orders_scaled).transform(olist_orders_scaled)

olist_orders_cat.select(categorical_cols + [col+"_ohe" for col in categorical_cols]).show(5, truncate=False)

In [0]:
# Створення часових ознак
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, weekofyear, hour, datediff

olist_orders_time = olist_orders_cat.withColumn("purchase_year", year("order_purchase_timestamp")) \
    .withColumn("purchase_month", month("order_purchase_timestamp")) \
    .withColumn("purchase_day", dayofmonth("order_purchase_timestamp")) \
    .withColumn("purchase_weekday", dayofweek("order_purchase_timestamp")) \
    .withColumn("purchase_weekofyear", weekofyear("order_purchase_timestamp")) \
    .withColumn("delivery_time_days_calc", datediff("order_delivered_customer_date", "order_purchase_timestamp"))

olist_orders_time.select("order_id", "order_purchase_timestamp", "delivery_time_days_calc").show(5)


In [0]:
# Аналіз кореляцій між числовими ознаками
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

assembler_corr = VectorAssembler(inputCols=numeric_cols, outputCol="corr_features")
olist_orders_corr = assembler_corr.transform(olist_orders_time)

corr_matrix = Correlation.corr(olist_orders_corr, "corr_features", "pearson").head()[0]
print("Кореляційна матриця (Pearson):")
print(corr_matrix)


In [0]:
# Вибір найбільш інформативних числових ознак
numeric_cols_filtered = [
    "purchase_year", "purchase_month",
    "purchase_day", "purchase_weekday", "delivery_time_days"
]

print("Фінальний список числових ознак після відбору:", numeric_cols_filtered)

In [0]:
# Агрегування даних по клієнту
from pyspark.sql.functions import avg, count

# Кількість замовлень на клієнта
olist_orders_agg = olist_orders_time.groupBy("customer_id").agg(
    count("order_id").alias("orders_per_customer"),
    avg("delivery_time_days_calc").alias("avg_delivery_days")
)

olist_orders_agg.show(5)

In [0]:
# Формування фінального набору даних
# Залишено основні ознаки
final_cols = ["order_id", "customer_id", "numeric_scaled"] + \
             [col+"_ohe" for col in categorical_cols] + \
             ["purchase_year", "purchase_month", "purchase_day", "purchase_weekday",
              "delivery_time_days_calc", "orders_per_customer", "avg_delivery_days"]

olist_orders_final = olist_orders_agg.join(olist_orders_time, on="customer_id", how="inner").select(final_cols)
olist_orders_final.show(5)


In [0]:
delta_output_path = "/Volumes/workspace/default/olist_delta_2"

olist_orders_final.write.format("delta").mode("overwrite").save(delta_output_path)
